In [2]:
!pip install librosa


  Using cached librosa-0.11.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached soundfile-0.13.1-py2.py3-none-macosx_11_0_arm64.whl.metadata (16 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
Using cached librosa-0.11.0-py3-none-any.whl (260 kB)
Using cached audioread-3.0.1-py3-none-any.whl (23 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 6.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 17.8 MB/s eta 0:00:00a 0:00:01
Using cached pooch-1.8.2-py3-none-any.whl (64 kB)
Using cached soundfile-0.13.1-py2.py3-none-macosx_11_0_arm64.whl (1.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [librosa]m7/9 [numba]ile]


In [5]:
# Audio processing and feature extraction
import librosa
import soundfile

# File and data handling
import os
import glob
import pickle

# Numerical operations
import numpy as np

# Machine learning and evaluation
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

print("All libraries imported successfully!")


All libraries imported successfully!


In [6]:
def extract_feature(file_name, mfcc, chroma, mel):
    X, sample_rate = librosa.load(file_name, res_type='scipy')

    if chroma:
        stft = np.abs(librosa.stft(X))
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
        result = np.hstack((result, mel))
    return result


In [7]:
emotions = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}
observed_emotions = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']


In [8]:
import glob

speech_files = glob.glob('/Users/dushyantyadav/Downloads/Audio_Speech_Actors_01-24/Actor*/**/*.wav', recursive=True)
song_files = glob.glob('/Users/dushyantyadav/Downloads/Audio_Song_Actors_01-24/Actor*/**/*.wav', recursive=True)
print("Speech files found:", len(speech_files))
print("Song files found:", len(song_files))


Speech files found: 1440
Song files found: 1012


In [9]:
def load_data(test_size=0.2):
    x, y = [], []
    # Pattern for speech files
    speech_files = glob.glob('/Users/dushyantyadav/Downloads/Audio_Speech_Actors_01-24/Actor*/**/*.wav', recursive=True)
    # Pattern for song files
    song_files = glob.glob('/Users/dushyantyadav/Downloads/Audio_Song_Actors_01-24/Actor*/**/*.wav', recursive=True)
    # Combine both file lists
    all_files = speech_files + song_files

    for file in all_files:
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)


In [170]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)


In [172]:
print((x_train.shape[0], x_test.shape[0]))

(1961, 491)


In [174]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [11]:
import os
import glob
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping


In [12]:
def extract_mfcc_sequence(file_path, n_mfcc=40, max_len=200):
    y, sr = librosa.load(file_path, res_type='scipy')
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    # Pad or truncate to fixed length for batching
    if mfcc.shape[1] < max_len:
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0,0),(0,pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    return mfcc.T  # Shape: (max_len, n_mfcc)


In [15]:
emotions = {
    '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
    '05': 'angry', '06': 'fearful', '07': 'disgust', '08': 'surprised'
}
observed_emotions = list(emotions.values())

def load_data_dl(test_size=0.2, max_len=200):
    x, y = [], []
    # Update these paths for your dataset
    speech_files = glob.glob('/Users/dushyantyadav/Downloads/Audio_Speech_Actors_01-24/Actor*/**/*.wav', recursive=True)
    song_files = glob.glob('/Users/dushyantyadav/Downloads/Audio_Song_Actors_01-24/Actor*/**/*.wav', recursive=True)
    all_files = speech_files + song_files
    for file in all_files:
        file_name = os.path.basename(file)
        emotion_code = file_name.split("-")[2]
        emotion = emotions.get(emotion_code)
        if emotion not in observed_emotions:
            continue
        try:
            mfcc_seq = extract_mfcc_sequence(file, max_len=max_len)
            x.append(mfcc_seq)
            y.append(emotion)
        except Exception as e:
            print(f"Error processing {file}: {e}")
    x = np.array(x)
    y = np.array(y)
    le = LabelEncoder()
    y_enc = le.fit_transform(y)
    y_cat = to_categorical(y_enc)
    x_train, x_test, y_train, y_test = train_test_split(
        x, y_cat, test_size=test_size, random_state=42, stratify=y
    )
    return x_train, x_test, y_train, y_test, le


In [16]:
def build_cnn_gru(input_shape, num_classes):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=5, activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(Conv1D(128, kernel_size=5, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(GRU(128, return_sequences=False))  # <-- GRU replaces LSTM
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [ ]:
# Load data
x_train, x_test, y_train, y_test, le = load_data_dl(test_size=0.2, max_len=200)
input_shape = x_train.shape[1:]  # (max_len, n_mfcc)
num_classes = y_train.shape[1]

# Build model
model = build_cnn_gru(input_shape, num_classes)

# Early stopping for better generalization
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train
history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop]
)

# Evaluate
loss, acc = model.evaluate(x_test, y_test)
print("Test accuracy:", acc)

from sklearn.metrics import classification_report, confusion_matrix
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred, target_names=le.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))


/Users/dushyantyadav/anaconda3/envs/tf_macos/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-06-21 20:38:23.074057: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-06-21 20:38:23.075069: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-06-21 20:38:23.075111: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-06-21 20:38:23.075680: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-06-21 20:38:23.076088: I tensorflow/core/common_runtime/pluggable_device/pluggable_

Epoch 1/100


2025-06-21 20:38:24.545889: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


62/62 ━━━━━━━━━━━━━━━━━━━━ 10s 66ms/step - accuracy: 0.1491 - loss: 2.2587 - val_accuracy: 0.1426 - val_loss: 2.3989
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.2286 - loss: 2.1821 - val_accuracy: 0.1752 - val_loss: 2.4319
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.2338 - loss: 1.9692 - val_accuracy: 0.2505 - val_loss: 1.9772
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.2499 - loss: 1.9902 - val_accuracy: 0.2464 - val_loss: 2.0008
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.2736 - loss: 1.8897 - val_accuracy: 0.2729 - val_loss: 2.0035
Epoch 6/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.2992 - loss: 1.8852 - val_accuracy: 0.2974 - val_loss: 1.9073
Epoch 7/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.3115 - loss: 1.7959 - val_accuracy: 0.3544 - val_loss: 1.6036
Epoch 8/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.3269 - loss: 1.7624 - val_accuracy: 0.3320 - val_

In [1]:
import os
import glob
import numpy as np
import librosa
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix


In [3]:
# Emotion mapping (adapt for your dataset)
emotions = {
    '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
    '05': 'angry', '06': 'fearful', '07': 'disgust', '08': 'surprised'
}
observed_emotions = list(emotions.values())
DATA_PATHS = [
    '/Users/dushyantyadav/Downloads/Audio_Speech_Actors_01-24/Actor*/**/*.wav',
    '/Users/dushyantyadav/Downloads/Audio_Song_Actors_01-24/Actor*/**/*.wav'
]


In [5]:
def add_noise(data, noise_factor=0.005):
    noise = np.random.randn(len(data))
    return data + noise_factor * noise

def shift(data, shift_max=0.2, shift_direction='both'):
    shift_amt = np.random.randint(int(len(data) * shift_max))
    if shift_direction == 'right':
        shift_amt = -shift_amt
    elif shift_direction == 'both':
        if np.random.randint(0, 2) == 1:
            shift_amt = -shift_amt
    return np.roll(data, shift_amt)


In [7]:
def extract_mfcc_sequence(file_path, n_mfcc=40, max_len=200):
    y, sr = librosa.load(file_path, res_type='scipy')
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    # Pad or truncate for batching
    if mfcc.shape[1] < max_len:
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0,0),(0,pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    return mfcc.T  # (max_len, n_mfcc)


In [9]:
def load_data_dl(test_size=0.2, max_len=200, augment=True):
    x, y = [], []
    files = []
    for path in DATA_PATHS:
        files.extend(glob.glob(path, recursive=True))
    for file in files:
        file_name = os.path.basename(file)
        emotion_code = file_name.split("-")[2]
        emotion = emotions.get(emotion_code)
        if emotion not in observed_emotions:
            continue
        try:
            # Original
            mfcc_seq = extract_mfcc_sequence(file, max_len=max_len)
            x.append(mfcc_seq)
            y.append(emotion)
            if augment:
                # Augmented: noise
                y_audio, sr = librosa.load(file, res_type='scipy')
                mfcc_noise = librosa.feature.mfcc(y=add_noise(y_audio), sr=sr, n_mfcc=40)
                if mfcc_noise.shape[1] < max_len:
                    pad_width = max_len - mfcc_noise.shape[1]
                    mfcc_noise = np.pad(mfcc_noise, pad_width=((0,0),(0,pad_width)), mode='constant')
                else:
                    mfcc_noise = mfcc_noise[:, :max_len]
                x.append(mfcc_noise.T)
                y.append(emotion)
                # Augmented: shift
                y_shift = shift(y_audio)
                mfcc_shift = librosa.feature.mfcc(y=y_shift, sr=sr, n_mfcc=40)
                if mfcc_shift.shape[1] < max_len:
                    pad_width = max_len - mfcc_shift.shape[1]
                    mfcc_shift = np.pad(mfcc_shift, pad_width=((0,0),(0,pad_width)), mode='constant')
                else:
                    mfcc_shift = mfcc_shift[:, :max_len]
                x.append(mfcc_shift.T)
                y.append(emotion)
        except Exception as e:
            print(f"Error processing {file}: {e}")
    x = np.array(x)
    y = np.array(y)
    le = LabelEncoder()
    y_enc = le.fit_transform(y)
    y_cat = to_categorical(y_enc)
    x_train, x_test, y_train, y_test = train_test_split(
        x, y_cat, test_size=test_size, random_state=42, stratify=y)
    return x_train, x_test, y_train, y_test, le, y


In [11]:
def build_cnn_lstm(input_shape, num_classes):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=5, activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(Conv1D(128, kernel_size=5, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(LSTM(128, return_sequences=False))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [13]:
# After loading data
x_train, x_test, y_train, y_test, le, y_all = load_data_dl(test_size=0.2, max_len=200, augment=True)
y_train_labels = np.argmax(y_train, axis=1)
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_labels), y=y_train_labels)
class_weight_dict = dict(enumerate(class_weights))
print("Class weights:", class_weight_dict)


Class weights: {0: 0.815410199556541, 1: 0.8145071982281284, 2: 1.5954446854663775, 3: 0.815410199556541, 4: 0.815410199556541, 5: 1.630820399113082, 6: 0.815410199556541, 7: 1.5954446854663775}


In [15]:
input_shape = x_train.shape[1:]  # (max_len, n_mfcc)
num_classes = y_train.shape[1]
model = build_cnn_lstm(input_shape, num_classes)

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=60,
    batch_size=32,
    callbacks=[early_stop],
    class_weight=class_weight_dict
)

# Evaluate
loss, acc = model.evaluate(x_test, y_test)
print("Test accuracy:", acc)

y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred, target_names=le.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))


2025-06-21 21:49:57.622698: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-06-21 21:49:57.622960: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-06-21 21:49:57.622987: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-06-21 21:49:57.623362: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-06-21 21:49:57.623824: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/60


2025-06-21 21:49:59.570151: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


184/184 [==============================] - 8s 31ms/step - loss: 2.0172 - accuracy: 0.2039 - val_loss: 1.8619 - val_accuracy: 0.2772
Epoch 2/60
184/184 [==============================] - 5s 26ms/step - loss: 1.7506 - accuracy: 0.3110 - val_loss: 1.6217 - val_accuracy: 0.3913
Epoch 3/60
184/184 [==============================] - 5s 26ms/step - loss: 1.4754 - accuracy: 0.4369 - val_loss: 1.4525 - val_accuracy: 0.4436
Epoch 4/60
184/184 [==============================] - 5s 26ms/step - loss: 1.3165 - accuracy: 0.4944 - val_loss: 1.2939 - val_accuracy: 0.5082
Epoch 5/60
184/184 [==============================] - 5s 26ms/step - loss: 1.2301 - accuracy: 0.5318 - val_loss: 1.2052 - val_accuracy: 0.5306
Epoch 6/60
184/184 [==============================] - 5s 26ms/step - loss: 1.1058 - accuracy: 0.5766 - val_loss: 1.0416 - val_accuracy: 0.6080
Epoch 7/60
184/184 [==============================] - 5s 26ms/step - loss: 0.9971 - accuracy: 0.6101 - val_loss: 1.0168 - val_accuracy: 0.6230
Epoch 8/60

In [17]:
def predict_emotion(file_path, model, le, max_len=200):
    mfcc_seq = extract_mfcc_sequence(file_path, max_len=max_len)
    mfcc_seq = np.expand_dims(mfcc_seq, axis=0)
    pred = model.predict(mfcc_seq)
    predicted_class = np.argmax(pred)
    return le.classes_[predicted_class]

# Example usage
test_file = '/Users/dushyantyadav/Downloads/Crema/1001_DFA_ANG_XX.wav'
print("Predicted emotion:", predict_emotion(test_file, model, le))


1/1 [==============================] - 1s 1s/step
Predicted emotion: surprised


In [19]:
def predict_emotion(file_path, model, le, max_len=200):
    mfcc_seq = extract_mfcc_sequence(file_path, max_len=max_len)
    mfcc_seq = np.expand_dims(mfcc_seq, axis=0)
    pred = model.predict(mfcc_seq)
    predicted_class = np.argmax(pred)
    return le.classes_[predicted_class]

# Example usage
test_file = '/Users/dushyantyadav/Downloads/Crema/1076_IEO_SAD_MD.wav'
print("Predicted emotion:", predict_emotion(test_file, model, le))

1/1 [==============================] - 0s 34ms/step
Predicted emotion: fearful


In [23]:

def predict_emotion(file_path, model, le, max_len=200):
    mfcc_seq = extract_mfcc_sequence(file_path, max_len=max_len)
    mfcc_seq = np.expand_dims(mfcc_seq, axis=0)
    pred = model.predict(mfcc_seq)
    predicted_class = np.argmax(pred)
    return le.classes_[predicted_class]

# Example usage
test_file = '/Users/dushyantyadav/Downloads/Crema/1082_IEO_FEA_MD.wav'
print("Predicted emotion:", predict_emotion(test_file, model, le))

1/1 [==============================] - 0s 62ms/step
Predicted emotion: fearful
